# Regresión Logística: Detección de SPAM

En este ejercicio se muestran los fundamentos de la Regresión Logística plamteando uno de los primeros problemas que fueron solucionados mediante el uso de técnicas de Machine Learning: Detección de SPAM.

## Enunciado del ejercicio 
Se propone la cosntrucción de un sistema de aprendizaje automatico capaz de predicir si un correo determinado corresponde a un correo SPAM o no, para esto se utilizara el siguiente DataSet: 

[DataSet]()

The corpus trec07p contains 75,419 messages:

25,220 Ham
50,199 SPAM

Theese messages contitue all the messages delivered to a particular server between these dates:

Sun, 8 Apr 2007 13:07:21 -0400
Fri, 6 Jul 2007 07:04:53 -0400

### 1.- Funciones Complementarias 

En este caso practico relacionado con la detectción de e-mails de SPAM, el DataSet del que se dispone. esa formado por e-mails, con sus corresponedientes cabeceras y campos adicionales. Por lo tanto requieren un preprocesamineto previo a ser ingeridos por el algoritmo de Machine Learning

In [ ]:
from utils.termal_control import iniciar_control
iniciar_control()

-->Control de hilos iniciado en segundo plano.


[thermal_control] Temp=35.0°C → 10 hilos
[thermal_control] Temp=35.6°C → 9 hilos
[thermal_control] Temp=35.1°C → 9 hilos
[thermal_control] Temp=34.8°C → 10 hilos
[thermal_control] Temp=35.1°C → 9 hilos
[thermal_control] Temp=35.1°C → 9 hilos
[thermal_control] Temp=35.1°C → 9 hilos
[thermal_control] Temp=35.0°C → 10 hilos
[thermal_control] Temp=35.0°C → 10 hilos
[thermal_control] Temp=35.1°C → 9 hilos
[thermal_control] Temp=35.1°C → 9 hilos
[thermal_control] Temp=35.6°C → 9 hilos
[thermal_control] Temp=36.1°C → 9 hilos
[thermal_control] Temp=36.9°C → 9 hilos
[thermal_control] Temp=37.1°C → 9 hilos
[thermal_control] Temp=37.1°C → 9 hilos
[thermal_control] Temp=36.9°C → 9 hilos
[thermal_control] Temp=36.8°C → 9 hilos
[thermal_control] Temp=36.1°C → 9 hilos
[thermal_control] Temp=36.0°C → 9 hilos
[thermal_control] Temp=36.8°C → 9 hilos
[thermal_control] Temp=36.8°C → 9 hilos
[thermal_control] Temp=36.1°C → 9 hilos
[thermal_control] Temp=35.9°C → 9 hilos
[thermal_control] Temp=35.6°C → 9 hi

In [1]:
# Esta clase facilita el procesamiento de correos Eleectronicos que poseen código HTML
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.fed = []

    def handle_data(self, d):
        self.fed.append(d)

    def get_data(self):
        return ''.join(self.fed)

In [2]:
# Esta funcion se encarga de eliminar los tags de HTML que se encuentran en el texto del e-mail.

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data() 

In [3]:
# EJemplo de elimiinacion de los tags HTML de un texto 
t = '<tr><td align="left"><a href="../../issues/21/16.html#article">Phrack Wordl News</a><td>'
strip_tags(t)

'Phrack Wordl News'

Ademas de eliminar los posibles tag o htmnl que se encuentren en el correo electtrónico deben realizarse otras acciones de preprocesamiento para evitar que los mensajes contengan ruido inecesario. Entre ellas se encuentra la eliminación de los signos de puntuación, eliminación de posibles campos de correo electrónico que no son relevantes o afijos de una palabra manteniendo únicamente la raiz de la misma (Steamming). La clase que se muestra a continuación realiza estas transformaciones.

In [4]:
import email
import string
import nltk


In [8]:
class Parser:
    
    def __init__(self):
        self.stemmer = nltk.PorterStemmer()
        self.stopwords = set(nltk.corpus.stopwords.words('english'))
        self.punctuation = list(string.punctuation)

    def parse(self, email_pat):
        """Parse an email."""
        with open(email_pat, errors = 'ignore') as e:
            msg = email.message_from_file(e)
        return None if not msg else self.get_email_content(msg)
    
    def get_email_content(self, msg):
        """Extract the email content."""
        subjetc  = self.tokenize(msg['Subject']) if msg['subjetc'] else []
        body = self.get_email_body(msg.get_payload(),
                                   msg.get_content_type())
        content_type = msg.get_content_type()

        # Returning the content of the email

        return {"subject":subjetc, 
                "body": body,
                "content_type": content_type}
    
    def get_email_body(self, payload, content_type):
        """Extract body of the email"""
        body = []
        if type(payload) is str and content_type == 'text/plain':
            return self.tokenize(payload)
        elif type(payload) is str and content_type == 'text/html':
            return self.tokenize(strip_tags(payload))
        elif type(payload) is list:
            for p in payload:
                body += self.get_email_body(p.get_payload(),
                                            p.get_content_type())
        return body
    
    def tokenize(self, text):
        """Transform a text string in tokens, perform two main actions
        clean the punctuation symbols and do stemming of the text."""

        for c in self.punctuation:
            text = text.replace(c, "")
        text = text.replace("\t"," ")
        text = text.replace("\n"," ")
        tokens = list(filter(None, text.split(" ")))

        #Stemming of the tokens 
        return [self.stemmer.stem(w) for w in tokens if w not in self.stopwords]

Lectura de un e-mail en formato Raw

In [11]:
inmail = open("datasets/trec07p/data/inmail.1").read()

print(inmail)

From RickyAmes@aol.com  Sun Apr  8 13:07:32 2007
Return-Path: <RickyAmes@aol.com>
Received: from 129.97.78.23 ([211.202.101.74])
	by speedy.uwaterloo.ca (8.12.8/8.12.5) with SMTP id l38H7G0I003017;
	Sun, 8 Apr 2007 13:07:21 -0400
Received: from 0.144.152.6 by 211.202.101.74; Sun, 08 Apr 2007 19:04:48 +0100
Message-ID: <WYADCKPDFWWTWTXNFVUE@yahoo.com>
From: "Tomas Jacobs" <RickyAmes@aol.com>
Reply-To: "Tomas Jacobs" <RickyAmes@aol.com>
To: the00@speedy.uwaterloo.ca
Subject: Generic Cialis, branded quality@ 
Date: Sun, 08 Apr 2007 21:00:48 +0300
X-Mailer: Microsoft Outlook Express 6.00.2600.0000
MIME-Version: 1.0
Content-Type: multipart/alternative;
	boundary="--8896484051606557286"
X-Priority: 3
X-MSMail-Priority: Normal
Status: RO
Content-Length: 988
Lines: 24

----8896484051606557286
Content-Type: text/html;
Content-Transfer-Encoding: 7Bit

<html>
<body bgcolor="#ffffff">
<div style="border-color: #00FFFF; border-right-width: 0px; border-bottom-width: 0px; margin-bottom: 0px;" align="